In [1]:
# import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random


In [2]:
df_all = pd.read_csv('../train_val_test_split/train_val_test_GoogleAudioSet.csv', index_col=0)
df_all = pd.concat([df_all, df_all], ignore_index=True)
df_all

file         source  \
0      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
1      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
2      ../data/interim/GoogleAudioSet_unbalanced_list...  Google_nature   
3      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
4      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
...                                                  ...            ...   
27329  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
27330  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
27331  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
27332  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
27333  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   

       category  weight  fold  
0             1       1     0  
1             1       1     8  
2             0       1     5  
3             1       1     1  
4             1       1     1  
...         ...     ...   ...  
27329         1       1     5  
27330         1       1     3  
27331         1       1     8  
27332         1       1     8  
27333         1       1     0  

[27334 rows x 5 columns]

In [3]:
embedding_raw_matrix = np.vstack((np.load('../data/processed/20230304/embedding_raw_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/embedding_raw_matrix.npy')))
embedding_bg_matrix = np.vstack((np.load('../data/processed/20230304/embedding_bg_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/embedding_bg_matrix.npy')))
embedding_fg_matrix = np.vstack((np.load('../data/processed/20230304/embedding_fg_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/embedding_fg_matrix.npy')))

In [4]:
vgg_raw_matrix = np.vstack((np.load('../data/processed/20230304/vgg_raw_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/vgg_raw_matrix.npy')))
vgg_bg_matrix = np.vstack((np.load('../data/processed/20230304/vgg_bg_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/vgg_bg_matrix.npy')))
vgg_fg_matrix = np.vstack((np.load('../data/processed/20230304/vgg_fg_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/vgg_fg_matrix.npy')))

In [5]:
mps_raw_matrix = np.vstack((np.load('../data/processed/20230304/mps_raw_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/mps_raw_matrix.npy')))
mps_bg_matrix = np.vstack((np.load('../data/processed/20230304/mps_bg_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/mps_bg_matrix.npy')))
mps_fg_matrix = np.vstack((np.load('../data/processed/20230304/mps_fg_matrix.npy'), np.load('../data/processed/20230310_dataAugmentation/mps_fg_matrix.npy')))

In [6]:
df_indices_raw = pd.concat([pd.read_csv('../data/processed/20230304/df_indices_raw.csv', index_col=0),pd.read_csv('../data/processed/20230310_dataAugmentation/df_indices_raw.csv', index_col=0)], ignore_index=True)
df_indices_bg = pd.concat([pd.read_csv('../data/processed/20230304/df_indices_bg.csv', index_col=0),pd.read_csv('../data/processed/20230310_dataAugmentation/df_indices_bg.csv', index_col=0)], ignore_index=True)
df_indices_fg = pd.concat([pd.read_csv('../data/processed/20230304/df_indices_fg.csv', index_col=0),pd.read_csv('../data/processed/20230310_dataAugmentation/df_indices_fg.csv', index_col=0)], ignore_index=True)

In [7]:
panns_clip_raw = np.vstack((np.load('../data/processed/20230304/panns_clip_raw.npy'), np.load('../data/processed/20230310_dataAugmentation/panns_clip_raw.npy')))
panns_clip_bg = np.vstack((np.load('../data/processed/20230304/panns_clip_bg.npy'), np.load('../data/processed/20230310_dataAugmentation/panns_clip_bg.npy')))
panns_clip_fg = np.vstack((np.load('../data/processed/20230304/panns_clip_fg.npy'), np.load('../data/processed/20230310_dataAugmentation/panns_clip_fg.npy')))
panns_embedding_raw = np.vstack((np.load('../data/processed/20230304/panns_embedding_raw.npy'), np.load('../data/processed/20230310_dataAugmentation/panns_embedding_raw.npy')))
panns_embedding_bg = np.vstack((np.load('../data/processed/20230304/panns_embedding_bg.npy'), np.load('../data/processed/20230310_dataAugmentation/panns_embedding_bg.npy')))
panns_embedding_fg = np.vstack((np.load('../data/processed/20230304/panns_embedding_fg.npy'), np.load('../data/processed/20230310_dataAugmentation/panns_embedding_fg.npy')))

In [8]:
index_train = df_all['fold'] < 8
index_valid = df_all['fold'] == 8
index_test = df_all['fold'] == 9

# PCA dimensional reduction

In [9]:
def get_transformer(n_components):
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    from sklearn.decomposition import PCA
    from sklearn.impute import SimpleImputer
    transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
        ('PCA', PCA(n_components = n_components, random_state=23)),
        ('scaler', StandardScaler())
    ])
    return transformer

In [10]:
def data_pca(data, index_train, index_valid, index_test, n_components):
    train_data = data[index_train]
    valid_data = data[index_valid]
    test_data = data[index_test]

    transformer = get_transformer(n_components = n_components)
    transformer.fit(train_data)
    train_data_pca = transformer.transform(train_data)
    valid_data_pca = transformer.transform(valid_data)
    test_data_pca = transformer.transform(test_data)
    return train_data_pca, valid_data_pca, test_data_pca

In [11]:
train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca = data_pca(mps_raw_matrix, index_train, index_valid, index_test, n_components=10)
train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca = data_pca(mps_bg_matrix, index_train, index_valid, index_test, n_components=10)
train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca = data_pca(mps_fg_matrix, index_train, index_valid, index_test, n_components=10)

In [12]:
train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca = data_pca(df_indices_raw.replace([np.inf, -np.inf], np.nan), index_train, index_valid, index_test, n_components=20)
train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca = data_pca(df_indices_bg.replace([np.inf, -np.inf], np.nan), index_train, index_valid, index_test, n_components=20)
train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca = data_pca(df_indices_fg.replace([np.inf, -np.inf], np.nan), index_train, index_valid, index_test, n_components=20)

In [13]:
train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca = data_pca(embedding_raw_matrix, index_train, index_valid, index_test, n_components=400)
train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca = data_pca(embedding_bg_matrix, index_train, index_valid, index_test, n_components=400)
train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca = data_pca(embedding_fg_matrix, index_train, index_valid, index_test, n_components=400)

In [14]:
train_vgg_raw_pca, valid_vgg_raw_pca, test_vgg_raw_pca = data_pca(vgg_raw_matrix, index_train, index_valid, index_test, n_components=128)
train_vgg_bg_pca, valid_vgg_bg_pca, test_vgg_bg_pca = data_pca(vgg_bg_matrix, index_train, index_valid, index_test, n_components=128)
train_vgg_fg_pca, valid_vgg_fg_pca, test_vgg_fg_pca = data_pca(vgg_fg_matrix, index_train, index_valid, index_test, n_components=128)

In [15]:
train_panns_clip_raw_pca, valid_panns_clip_raw_pca, test_panns_clip_raw_pca = data_pca(panns_clip_raw, index_train, index_valid, index_test, n_components=180)
train_panns_clip_bg_pca, valid_panns_clip_bg_pca, test_panns_clip_bg_pca = data_pca(panns_clip_bg, index_train, index_valid, index_test, n_components=180)
train_panns_clip_fg_pca, valid_panns_clip_fg_pca, test_panns_clip_fg_pca = data_pca(panns_clip_fg, index_train, index_valid, index_test, n_components=180)

train_panns_embedding_raw_pca, valid_panns_embedding_raw_pca, test_panns_embedding_raw_pca = data_pca(panns_embedding_raw, index_train, index_valid, index_test, n_components=400)
train_panns_embedding_bg_pca, valid_panns_embedding_bg_pca, test_panns_embedding_bg_pca = data_pca(panns_embedding_bg, index_train, index_valid, index_test, n_components=400)
train_panns_embedding_fg_pca, valid_panns_embedding_fg_pca, test_panns_embedding_fg_pca = data_pca(panns_embedding_fg, index_train, index_valid, index_test, n_components=400)

In [16]:
y_train = df_all.loc[index_train,'category']
y_valid = df_all.loc[index_valid,'category']
y_test = df_all.loc[index_test,'category']

In [17]:
import pickle
aggregated_data = [train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca,
                   train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca,
                   train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca,
                   train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca,
                   train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca,
                   train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca,
                   train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca,
                   train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca,
                   train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca,
                   train_vgg_raw_pca, valid_vgg_raw_pca, test_vgg_raw_pca,
                   train_vgg_bg_pca, valid_vgg_bg_pca, test_vgg_bg_pca,
                   train_vgg_fg_pca, valid_vgg_fg_pca, test_vgg_fg_pca,
                   train_panns_clip_raw_pca, valid_panns_clip_raw_pca, test_panns_clip_raw_pca,
                   train_panns_clip_bg_pca, valid_panns_clip_bg_pca, test_panns_clip_bg_pca,
                   train_panns_clip_fg_pca, valid_panns_clip_fg_pca, test_panns_clip_fg_pca,
                   train_panns_embedding_raw_pca, valid_panns_embedding_raw_pca, test_panns_embedding_raw_pca,
                   train_panns_embedding_bg_pca, valid_panns_embedding_bg_pca, test_panns_embedding_bg_pca,
                   train_panns_embedding_fg_pca, valid_panns_embedding_fg_pca, test_panns_embedding_fg_pca,
                  y_train, y_valid, y_test]
pickle.dump(aggregated_data, open('../data/processed/20230310_dataAugmentation/aggregated_data', 'wb' ) )